In [345]:
import pandas as pd
import numpy as np
from functools import reduce
from IPython.display import display
# from sklearn.impute import KNNImputer # nie używam
# from sklearn.neighbors import base
import sklearn
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

In [346]:
vaccinationsDF = pd.read_csv('dataFiles/vaccinations.csv', sep=',', header=0, na_filter=False)
casesDF = pd.read_csv('dataFiles/covidCases.csv', sep=',', header=0, na_filter=False)
iso_codesDF = pd.read_csv('dataFiles/iso_codes.csv', sep=',', header=1, na_filter=False)

In [347]:
casesDF.rename(columns={'Date_reported':'date', 'Country':'location', 'Country_code': 'iso2Code'}, inplace=True)

vaccinationsDF.rename(columns={'iso_code': 'iso3Code'}, inplace=True)
vaccinationsDF.drop(vaccinationsDF[(vaccinationsDF['iso3Code'].apply(len) != 3)].index, inplace=True)
vaccinationsDF.reset_index(drop=True)

iso_codesDF.drop(iso_codesDF.columns[[1, 2, 5, 6, 7]], axis=1, inplace=True)
iso_codesDF.rename(columns={'Country name':'location', 'Alpha-2 code':'iso2Code', 'Alpha-3 code':'iso3Code'}, inplace=True)
iso_codesDF = iso_codesDF.astype(str)
iso_codesDF.drop(iso_codesDF[(iso_codesDF['iso2Code'].apply(len) != 2) | (iso_codesDF['iso3Code'].apply(len) != 3)].index, inplace=True)
iso_codesDF = iso_codesDF.reset_index(drop=True)

In [348]:
tmp_df = reduce(lambda left, right: pd.merge(left, right, on=['iso3Code'], how='inner'), [vaccinationsDF, iso_codesDF])
df = reduce(lambda left, right: pd.merge(left, right, on=['iso2Code', 'date'], how='outer'), [tmp_df, casesDF])

In [349]:
df['date'] = pd.to_datetime(df['date'], format='%Y.%m.%d').apply(lambda x: x.toordinal())

In [350]:
df = df.drop(['location_x', 'location_y', 'WHO_region', 'iso3Code', 'total_vaccinations_per_hundred', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred', 'daily_vaccinations_per_million', 'daily_people_vaccinated_per_hundred', 'daily_vaccinations_raw'], axis=1)

In [351]:
df = df.dropna(subset=['location'])
df = df.drop(['location'], axis=1)
df = df.reset_index(drop=True)

In [352]:
# replace empty values with nam
df = df.replace(r'^\s*$', np.nan, regex=True)

In [353]:
# Convert to int
# Somehow function "remove()" does not work
df_numeric_col = [x for x in df.columns.tolist() if x != 'iso2Code']
df[df_numeric_col] = df[df_numeric_col].astype('Int64').astype('Float64')

In [354]:
def undummify(df, prefix_sep="_"):
    cols2collapse = {
        item.split(prefix_sep)[0]: (prefix_sep in item) for item in df.columns
    }
    series_list = []
    for col, needs_to_collapse in cols2collapse.items():
        if needs_to_collapse:
            undummified = (
                df.filter(like=col)
                .idxmax(axis=1)
                .apply(lambda x: x.split(prefix_sep, maxsplit=1)[1])
                .rename(col)
            )
            series_list.append(undummified)
        else:
            series_list.append(df[col])
    undummified_df = pd.concat(series_list, axis=1)
    return undummified_df

In [355]:
selected_countries = ['PL', 'DE', 'NO', 'BE', 'GB', 'UA']
df_trimmed = df[df['iso2Code'].isin(selected_countries)]

In [356]:
df_dummies = pd.get_dummies(df_trimmed)
df_dummies =df_dummies.fillna(-1)

In [357]:
# display(df_dummies)
# df_numeric_col = [x for x in df.columns.tolist() if x != 'iso2Code']
# df_dummies.dtypes

# df = df.where(df.isna, 'NaN', df).astype(str)

# df_dummies[df_numeric_col] = df_dummies[df_numeric_col].dtype(str).fillna(r'NaN')

,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations,daily_people_vaccinated,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,iso2Code_BE,iso2Code_DE,iso2Code_GB,iso2Code_NO,iso2Code_PL,iso2Code_UA
10101,737787.0,340.0,340.0,-1.0,-1.0,-1.0,-1.0,530.0,641523.0,61.0,19534.0,1,0,0,0,0,0
10102,737788.0,380.0,379.0,11.0,-1.0,40.0,39.0,2641.0,644164.0,63.0,19597.0,1,0,0,0,0,0
10103,737789.0,912.0,910.0,21.0,-1.0,286.0,285.0,2270.0,646434.0,77.0,19674.0,1,0,0,0,0,0
10104,737790.0,960.0,958.0,21.0,-1.0,207.0,206.0,1980.0,648414.0,75.0,19749.0,1,0,0,0,0,0
10105,737791.0,977.0,975.0,21.0,-1.0,159.0,159.0,1626.0,650040.0,88.0,19837.0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225219,738401.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,5056378.0,0.0,108841.0,0,0,0,0,0,1
225220,738402.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,5056378.0,0.0,108841.0,0,0,0,0,0,1
225221,738403.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,16155.0,5072533.0,44.0,108885.0,0,0,0,0,0,1
225222,738404.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,5072533.0,0.0,108885.0,0,0,0,0,0,1


date                       Float64
total_vaccinations         Float64
people_vaccinated          Float64
people_fully_vaccinated    Float64
total_boosters             Float64
daily_vaccinations         Float64
daily_people_vaccinated    Float64
New_cases                  Float64
Cumulative_cases           Float64
New_deaths                 Float64
Cumulative_deaths          Float64
iso2Code_BE                  uint8
iso2Code_DE                  uint8
iso2Code_GB                  uint8
iso2Code_NO                  uint8
iso2Code_PL                  uint8
iso2Code_UA                  uint8
dtype: object

Completion of empty data using ML

In [362]:
df_dummies_columns = df_dummies.columns.tolist()
df_array = df_dummies.to_numpy()
#df_array = df_dummies.values
# [0 if math.isnan(x) else x for x in listname]
# xd = [[-1 if math.isnan(v) else v for v in l] for l in df_array] # 'NaN' if math.isnan(v) for l
# display(xd)

In [363]:
# display(df_array)
print(df_array)

[[737787.0 340.0 340.0 ... 0 0 0]
 [737788.0 380.0 379.0 ... 0 0 0]
 [737789.0 912.0 910.0 ... 0 0 0]
 ...
 [738403.0 -1.0 -1.0 ... 0 0 1]
 [738404.0 -1.0 -1.0 ... 0 0 1]
 [738405.0 -1.0 -1.0 ... 0 0 1]]


In [365]:
RF_transformator = MissForest(random_state=0, missing_values=-1, max_features='sqrt', criterion='squared_error', class_weight='balanced_subsample')
df_array_transformed = RF_transformator.fit_transform(df_array)

C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\U

Iteration: 0


C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\U

Iteration: 1


C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\U

...

Iteration: 4


C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\U

Iteration: 5


C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\Users\HP\PycharmProjects\covidAnalysis\venv\lib\site-packages\sklearn\ensemble\_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
C:\U

Iteration: 6


In [366]:
df_dum_transformed = pd.DataFrame(data=df_array_transformed, columns=df_dummies_columns)
display(df_dum_transformed)

,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations,daily_people_vaccinated,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,iso2Code_BE,iso2Code_DE,iso2Code_GB,iso2Code_NO,iso2Code_PL,iso2Code_UA
0,737787.0,340.00,340.00,11364.88,760599.79,1640.31,956.90,530.0,641523.0,61.0,19534.0,1.0,0.0,0.0,0.0,0.0,0.0
1,737788.0,380.00,379.00,11.00,1094813.20,40.00,39.00,2641.0,644164.0,63.0,19597.0,1.0,0.0,0.0,0.0,0.0,0.0
2,737789.0,912.00,910.00,21.00,1088175.68,286.00,285.00,2270.0,646434.0,77.0,19674.0,1.0,0.0,0.0,0.0,0.0,0.0
3,737790.0,960.00,958.00,21.00,1104734.27,207.00,206.00,1980.0,648414.0,75.0,19749.0,1.0,0.0,0.0,0.0,0.0,0.0
4,737791.0,977.00,975.00,21.00,1109980.91,159.00,159.00,1626.0,650040.0,88.0,19837.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5869,738401.0,43432363.93,21106980.11,19749330.31,7145008.85,48604.99,11307.34,0.0,5056378.0,0.0,108841.0,0.0,0.0,0.0,0.0,0.0,1.0
5870,738402.0,43432363.93,21106980.11,19749330.31,7145008.85,48604.99,11307.34,0.0,5056378.0,0.0,108841.0,0.0,0.0,0.0,0.0,0.0,1.0
5871,738403.0,42996668.97,21163046.23,19898951.09,6704275.25,58165.61,13086.64,16155.0,5072533.0,44.0,108885.0,0.0,0.0,0.0,0.0,0.0,1.0
5872,738404.0,43433895.08,21107313.88,19749330.31,7145008.85,48604.99,11179.27,0.0,5072533.0,0.0,108885.0,0.0,0.0,0.0,0.0,0.0,1.0


In [376]:
df_transformed = undummify(df_dum_transformed, prefix_sep='Code_')
df_transformed.rename(columns={'iso2': 'iso2Code'}, inplace=True)

In [377]:
display(df_transformed.head(15))

,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations,daily_people_vaccinated,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,iso2
0,737787.0,340.0,340.0,11364.88,760599.79,1640.31,956.9,530.0,641523.0,61.0,19534.0,BE
1,737788.0,380.0,379.0,11.00,1094813.20,40.00,39.0,2641.0,644164.0,63.0,19597.0,BE
2,737789.0,912.0,910.0,21.00,1088175.68,286.00,285.0,2270.0,646434.0,77.0,19674.0,BE
3,737790.0,960.0,958.0,21.00,1104734.27,207.00,206.0,1980.0,648414.0,75.0,19749.0,BE
4,737791.0,977.0,975.0,21.00,1109980.91,159.00,159.0,1626.0,650040.0,88.0,19837.0,BE
5,737792.0,1002.0,1000.0,34.00,1062014.32,132.00,132.0,469.0,650509.0,51.0,19888.0,BE
6,737793.0,1036.0,1034.0,34.00,1070509.40,116.00,116.0,1051.0,651560.0,64.0,19952.0,BE
7,737794.0,1109.0,1106.0,50.00,983001.86,110.00,109.0,732.0,652292.0,42.0,19994.0,BE
8,737795.0,2902.0,2896.0,79.00,1024511.86,360.00,360.0,2907.0,655199.0,59.0,20053.0,BE
9,737796.0,6933.0,6926.0,108.00,781851.69,860.00,859.0,2625.0,657824.0,63.0,20116.0,BE


In [ ]:
# df.to_csv('merged.csv', header=True, sep=';')

In [ ]:
# df_transformed = pd.Dataframe(df_array_transformed).transpose()
# df_transformed.columns = df_dummies_columns

In [ ]:
"""

# display(casesDF['location'].where(casesDF['iso2Code'].isnull()).unique())
# display(casesDF['iso2Code'].where(casesDF['location'].isnull()).unique())
# display(df.loc[df['iso2Code'] == 'XA'])
# display(df.isnull().sum())
# display(df.count())
# display(df[df['iso3Code'].isna()])
display(df.date)

display(len(df.location_x.unique()))
display(len(df.location_y.unique()))
display(len(df.location.unique()))
# display(casesDF.tail(5))
# display(vaccinationsDF.tail(5))

# display(vaccinationsDF)
display(iso_codesDF.location.unique().size)
display(len(vaccinationsDF.iso3Code.unique()))
# display(df['location'].where(['iso2Code'].isnull()))
display(df.loc[df['location_x'].isnull()])
display(casesDF.count())


def common_member(a, b):
    result = [i for i in a if i not in b]
    return len(result)

# display(len(common_member(casesDF.location.unique(), vaccinationsDF.location.unique())))

# display(common_member(casesDF.iso2Code.unique(), iso_codesDF.iso2Code.unique()))
# display(common_member(vaccinationsDF.iso3Code.unique(), iso_codesDF.iso3Code.unique()))
# display(casesDF.iso2Code.unique())
# display(iso_codesDF.loc[iso_codesDF['location'] == 'Namibia'])
display(common_member(df.location.unique(), df.location_x.unique()))
"""